Usecase test 1: demonstrating that genes given in an article's abstract are found to be amoung the highest significance genes linked to that article in the KG. 

1. Retrieving all named genes from the 10 articles first retrieved from Pubmed (from the stored metadata file)

In [9]:
#import scispacy
import spacy
import csv
import re
spacy.load("en_core_web_sm")
#from spacy.lang.en import English
#retrieve data from the metadata file
def get_csv_column(file_path, column_name):
    data = []
    with open(file_path, 'r') as csvfile:
        csvreader = csv.DictReader(csvfile)
        for row in csvreader:
            data.append(row[column_name])
    return data


def process_csv(file_path, search_terms, search_column, return_column):
    nlp = spacy.load("en_core_web_sm")
    results = []
    search_terms_lower = [term.lower() for term in search_terms]
    with open(file_path, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            search_text = row[search_column]
            return_value = row[return_column]
            doc = nlp(search_text)
            matched_terms = set(token.text for token in doc 
                                if token.text.lower() in search_terms_lower 
                                and not token.is_punct
                                and token.pos_ != "VERB")
            if matched_terms:
                results.append({
                    'matched_terms': list(matched_terms),
                    'return_value': return_value
                })
    return results


abstract_file = 'data/asd_article_metadata.csv'
gene_file = 'gene_list.csv'     #list of gene names taken from Ensembl 
column_name = 'Gene name'
gene_list = get_csv_column(gene_file, column_name)
search_column = 'abstract'      #will search through the columns of abstracts
return_column = 'pmid'           #will return the associated article pmid

results = process_csv(abstract_file, gene_list, search_column, return_column)

for result in results:
    print(f"Article PMID: {result['return_value']}")
    print(f"Matched gene terms: {result['matched_terms']}")
    
    print()

Article PMID: 36213201
Matched gene terms: ['set']

Article PMID: 36688057
Matched gene terms: ['CD14']

Article PMID: 37381037
Matched gene terms: ['ATRX', 'set', 'TEs', 'impact']

Article PMID: 38781372
Matched gene terms: ['GRN']



37381037 - contains ATRX


2. Querying the KG to retrieve the most significant genes within the datasets for each article: 

In [23]:
import rdflib
#from rdflib.plugins.sparql import prepareQuery
import requests

filename = "test_graph.nt"
g = rdflib.Graph()
g.parse(filename, format="nt")
#print(len(g))
test_query = """
    PREFIX EDAM: <http://edamontology.org/>
    PREFIX RDF: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX DCT: <http://purl.org/dc/terms/>
    PREFIX BIOLINK: <https://w3id.org/biolink/vocab/>
    PREFIX ENSEMBL: <http://identifiers.org/ensembl/>
    PREFIX NCBIGENE: <http://identifiers.org/ncbigene/>
    
    SELECT DISTINCT ?pmid ?gene ?usedPredicate ?logfold ?pvalue
    WHERE {
        VALUES ?genePredicate {
            BIOLINK:symbol 
            ENSEMBL:id 
            NCBIGENE:id } .
        ?pmid (<>|!<>)* ?gene .
        ?pmid (<>|!<>)* ?logfold .
        ?pmid (<>|!<>)* ?pvalue .
        ?pmid RDF:type DCT:identifier .
        ?subject EDAM:data_3754 ?logfold .
        ?subject EDAM:data_2082 ?pvalue .
        FILTER (?pvalue < 0.05) .
        BIND(?genePredicate AS ?usedPredicate)
    }
    LIMIT 10
    """
results = g.query(test_query)
for row in results:
    print(row)
    #print(f"{row.pmid} | {row.gene} | {row.usedPredicate} | {row.logfold} |{row.pvalue}")
    # Convert the row to a dictionary
    """row_dict = row.asdict()
    
    # Access the five variables
    a = row_dict['a']
    b = row_dict['b']
    c = row_dict['c']
    d = row_dict['d']
    e = row_dict['e']
    
    # Print all five variables
    print(f"{a} | {b} | {c} | {d} | {e}")"""

#        FILTER(REGEX(?pvalue), "p[-\\s]?value|pvalue") .
#        FILTER (xsd:float(?p_output) < 0.05)  
#   FILTER regex(?pvalue, "pvalue", "i")
#    ORDER BY ASC(?p_output)


KeyboardInterrupt: 